# Подготовка данных

    Подготовим данные для их дальнейшего анализа и использования в моделях

### Imports

In [35]:
import pandas as pd
import re
import string

from nltk.corpus import stopwords
from nltk import download

import pymystem3.mystem as mystem

pd.set_option('display.max_colwidth', None)

Устанавливаем лемматизатор

In [2]:
mystem.MYSTEM_DIR = './mystem'
mystem.MYSTEM_BIN = './mystem/mystem.exe'

mystem.autoinstall()

In [3]:
%%html
<style>
/* Any CSS style can go in here. */
.dataframe th {
    font-size: 16px;
}
.dataframe td {
    font-size: 14px;
}
</style>

## Загрузка данных

In [4]:
train_ds = pd.read_csv('data/dataset/train.tsv', sep='\t')
test_ds = pd.read_csv('data/dataset/test.tsv', sep='\t')

In [5]:
train_ds.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмешательство в американские выборы,1
1,Агент Кокорина назвал езду по встречке житейской историей,0
2,Госдума рассмотрит возможность введения секретных статей Уголовного кодекса,1
3,ФАС заблокировала поставку скоростных трамваев для Москвы,0
4,Против Навального завели дело о недоносительстве на Волкова,1


In [6]:
test_ds.head()

,title,is_fake
0,"Роскомнадзор представил реестр сочетаний цветов, нежелательных в России",0
1,Ночью под Минском на президентской горе Беларашмор (Пик Демократии) внезапно появилось лицо Николая Лукашенко,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудностях работы с Сергеем Шойгу,0
3,"Сельская церковь, собравшая рекордно низкое количество пожертвований, сгорела от тройного попадания молнии",0
4,Акции Google рухнули после объявления о перезапуске RuTube,0


In [7]:
train_size = train_ds.shape[0]
test_size = train_ds.shape[0]

In [8]:
full_ds = pd.concat([train_ds, test_ds])

    Объединяем таблицы, чтобы применять изменения ко всему набору данных

In [9]:
train_ds.shape, test_ds.shape, full_ds.shape

((5758, 2), (1000, 2), (6758, 2))

## Пунктуация
    Удалим знаки препинания и другие лишние символы, а также разобьём предложение на слова для дальнейшей обработки

In [10]:
full_ds['splitted'] = full_ds['title'].apply(lambda x: re.split(f'[\s{string.punctuation}]', x))

In [11]:
full_ds.head()

,title,is_fake,splitted
0,Москвичу Владимиру Клутину пришёл счёт за вмешательство в американские выборы,1,"[Москвичу, Владимиру, Клутину, пришёл, счёт, за, вмешательство, в, американские, выборы]"
1,Агент Кокорина назвал езду по встречке житейской историей,0,"[Агент, Кокорина, назвал, езду, по, встречке, житейской, историей]"
2,Госдума рассмотрит возможность введения секретных статей Уголовного кодекса,1,"[Госдума, рассмотрит, возможность, введения, секретных, статей, Уголовного, кодекса]"
3,ФАС заблокировала поставку скоростных трамваев для Москвы,0,"[ФАС, заблокировала, поставку, скоростных, трамваев, для, Москвы]"
4,Против Навального завели дело о недоносительстве на Волкова,1,"[Против, Навального, завели, дело, о, недоносительстве, на, Волкова]"


## Приведение в нижний регистр 

In [12]:
full_ds['lower_case'] = full_ds['splitted'].apply(lambda x: [i.lower() for i in x])

In [13]:
full_ds.head()

,title,is_fake,splitted,lower_case
0,Москвичу Владимиру Клутину пришёл счёт за вмешательство в американские выборы,1,"[Москвичу, Владимиру, Клутину, пришёл, счёт, за, вмешательство, в, американские, выборы]","[москвичу, владимиру, клутину, пришёл, счёт, за, вмешательство, в, американские, выборы]"
1,Агент Кокорина назвал езду по встречке житейской историей,0,"[Агент, Кокорина, назвал, езду, по, встречке, житейской, историей]","[агент, кокорина, назвал, езду, по, встречке, житейской, историей]"
2,Госдума рассмотрит возможность введения секретных статей Уголовного кодекса,1,"[Госдума, рассмотрит, возможность, введения, секретных, статей, Уголовного, кодекса]","[госдума, рассмотрит, возможность, введения, секретных, статей, уголовного, кодекса]"
3,ФАС заблокировала поставку скоростных трамваев для Москвы,0,"[ФАС, заблокировала, поставку, скоростных, трамваев, для, Москвы]","[фас, заблокировала, поставку, скоростных, трамваев, для, москвы]"
4,Против Навального завели дело о недоносительстве на Волкова,1,"[Против, Навального, завели, дело, о, недоносительстве, на, Волкова]","[против, навального, завели, дело, о, недоносительстве, на, волкова]"


## Лемматизация

    Данная операция приводит слова в их начальную форму, например существительное будет приведено в именительный падеж. 
    Выполняем данную операцию для того, чтобы слова в разных формах не раздували огромный словарь в будущем

In [22]:
lemmatizer = mystem.Mystem(grammar_info=False, disambiguation=False, entire_input=False)

In [23]:
%time full_ds['lemmatized'] = full_ds['lower_case'].apply(lambda x: lemmatizer.lemmatize(' '.join(x)))

Wall time: 8min 34s


In [24]:
full_ds.head()

,title,is_fake,splitted,lower_case,lemmatized
0,Москвичу Владимиру Клутину пришёл счёт за вмешательство в американские выборы,1,"[Москвичу, Владимиру, Клутину, пришёл, счёт, за, вмешательство, в, американские, выборы]","[москвичу, владимиру, клутину, пришёл, счёт, за, вмешательство, в, американские, выборы]","[москвич, владимир, клутин, приходить, счет, за, вмешательство, в, американский, выборы]"
1,Агент Кокорина назвал езду по встречке житейской историей,0,"[Агент, Кокорина, назвал, езду, по, встречке, житейской, историей]","[агент, кокорина, назвал, езду, по, встречке, житейской, историей]","[агент, кокорин, называть, езда, по, встречок, житейский, история]"
2,Госдума рассмотрит возможность введения секретных статей Уголовного кодекса,1,"[Госдума, рассмотрит, возможность, введения, секретных, статей, Уголовного, кодекса]","[госдума, рассмотрит, возможность, введения, секретных, статей, уголовного, кодекса]","[госдума, рассматривать, возможность, введение, секретный, статья, уголовный, кодекс]"
3,ФАС заблокировала поставку скоростных трамваев для Москвы,0,"[ФАС, заблокировала, поставку, скоростных, трамваев, для, Москвы]","[фас, заблокировала, поставку, скоростных, трамваев, для, москвы]","[фас, заблокировать, поставка, скоростной, трамвай, для, москва]"
4,Против Навального завели дело о недоносительстве на Волкова,1,"[Против, Навального, завели, дело, о, недоносительстве, на, Волкова]","[против, навального, завели, дело, о, недоносительстве, на, волкова]","[против, навальный, заводить, дело, о, недоносительство, на, волков]"


## Удаление стоп-слов

    Удалим стоп-слова(слова, которые часто встречаются в языке и не носят большой смысловой нагрузки), чтобы сократить 
    кол-во слов, используемых при тренировке, что ускорит обучение 

In [36]:
download("stopwords") # Загружаем стоп-слова

[nltk_data] Downloading package stopwords to C:\Users\Матвей
[nltk_data]     Иванов\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [40]:
full_ds['cleaned'] = full_ds['lemmatized'].apply(lambda x: [i for i in x if i not in stopwords.words('russian')])

In [42]:
full_ds.head()

,title,is_fake,splitted,lower_case,lemmatized,cleaned
0,Москвичу Владимиру Клутину пришёл счёт за вмешательство в американские выборы,1,"[Москвичу, Владимиру, Клутину, пришёл, счёт, за, вмешательство, в, американские, выборы]","[москвичу, владимиру, клутину, пришёл, счёт, за, вмешательство, в, американские, выборы]","[москвич, владимир, клутин, приходить, счет, за, вмешательство, в, американский, выборы]","[москвич, владимир, клутин, приходить, счет, вмешательство, американский, выборы]"
1,Агент Кокорина назвал езду по встречке житейской историей,0,"[Агент, Кокорина, назвал, езду, по, встречке, житейской, историей]","[агент, кокорина, назвал, езду, по, встречке, житейской, историей]","[агент, кокорин, называть, езда, по, встречок, житейский, история]","[агент, кокорин, называть, езда, встречок, житейский, история]"
2,Госдума рассмотрит возможность введения секретных статей Уголовного кодекса,1,"[Госдума, рассмотрит, возможность, введения, секретных, статей, Уголовного, кодекса]","[госдума, рассмотрит, возможность, введения, секретных, статей, уголовного, кодекса]","[госдума, рассматривать, возможность, введение, секретный, статья, уголовный, кодекс]","[госдума, рассматривать, возможность, введение, секретный, статья, уголовный, кодекс]"
3,ФАС заблокировала поставку скоростных трамваев для Москвы,0,"[ФАС, заблокировала, поставку, скоростных, трамваев, для, Москвы]","[фас, заблокировала, поставку, скоростных, трамваев, для, москвы]","[фас, заблокировать, поставка, скоростной, трамвай, для, москва]","[фас, заблокировать, поставка, скоростной, трамвай, москва]"
4,Против Навального завели дело о недоносительстве на Волкова,1,"[Против, Навального, завели, дело, о, недоносительстве, на, Волкова]","[против, навального, завели, дело, о, недоносительстве, на, волкова]","[против, навальный, заводить, дело, о, недоносительство, на, волков]","[против, навальный, заводить, дело, недоносительство, волков]"


### Сохранение данных

In [60]:
full_ds['cleaned'] = full_ds.cleaned.apply(lambda x: ' '.join(x))

In [61]:
new_train_ds = full_ds[['cleaned', 'is_fake']].iloc[:train_size]
new_test_ds = full_ds[['cleaned', 'is_fake']].iloc[train_size:]

In [62]:
train_ds.iloc[-1]

title      Бывший тренер Локомотива возглавил нальчикский Спартак
is_fake                                                         0
Name: 5757, dtype: object

In [63]:
new_train_ds.iloc[-1]

cleaned    бывший тренер локомотив возглавлять нальчикский спартак
is_fake                                                          0
Name: 5757, dtype: object

In [64]:
test_ds.iloc[0]

title      Роскомнадзор представил реестр сочетаний цветов, нежелательных в России
is_fake                                                                          0
Name: 0, dtype: object

In [65]:
new_test_ds.iloc[0]

cleaned    роскомнадзор представлять реестр сочетание цветок нежелательный россия
is_fake                                                                         0
Name: 0, dtype: object

In [66]:
new_train_ds.to_csv('data/c_train.csv', index=False)
new_test_ds.to_csv('data/c_test.csv', index=False)